## Project Description

MLB Advanced Media, as stated in a job description for which I was intrigued by, was looking to develop insights into predictability of a hit based on data acquired through their Statcast tool. Statcast is a high-speed, high-accuracy device that tracks ball and player movements. 

The findings of this task would be for use by analysts and commentators during game broadcasts. The problem statement for the specfic prediction I undertook is:

Based on the ballistics of the pitch and the ball hit into play, what is the likelihood it results in a hit.

## Notebook Description

9\. **Select important featurs** and then rerun models again.

- Deskew data using `box_cox()`
- Apply `MinMaxScaler()` to data
- Using `SelectFromModel()` with a `LogisticRegression()` estimator and an L1 penalty, select important features
- Define function to train and test a model using specified predictors and targets
- Train model on deskewed, normalized, and feature optimized data
- Models used:
    - K Nearest Neighbors
    - Logistic Regression
    - Decision Tree Classifier
    - Random Forest Classifier
- Results:
    - As suspected, while normalizing the data in the earlier step drastically lowered the test score for kneighbors, after applying feature selection the test accuracy score has now improved to around 80% with significantly lower dimensionality. Random Forest is also around 80% accuracy.

|   Model Name   |   Test Score  |   Train Score   |
| -----------|:---------------:|--------------:|
| K Nearest Neighbors | 0.8005 | 0.8596 |
| Logistic Regression | 0.7451 | 0.7485 |
| Decision Tree Classifier | 0.7638 | 0.9998 |
| Random Forest Classifier | 0.8010 | 0.9834 |

### Initialize packages and read in pickled data

In [3]:
# ! pip install scrapy
# ! pip install psycopg2
# ! pip install sqlalchemy
# ! pip install missingno --quiet
# ! pip install scipy

In [4]:
% run __init__.py

In [5]:
cd ..

/home/jovyan


In [6]:
df_model = pd.read_pickle('data/df_model.p')

### Set up target and predictors

In [8]:
df_model.drop('player_id', axis=1, inplace=True)

In [9]:
target = df_model['hit_flag']
predictors = df_model.drop('hit_flag', axis=1)

BoxCox requires all positive values, so I'll start this workflow by using a `MinMaxScaler` on my data

### `MinMaxScaler`

In [10]:
df_model_proc_all = predictors.copy()

In [11]:
min_max = MinMaxScaler(feature_range=(1E-10,1))

In [12]:
df_model_mm = pd.DataFrame(min_max.fit_transform(df_model_proc_all), 
                           index=df_model_proc_all.index, 
                           columns=df_model_proc_all.columns)

### Skew-Normalize Features

#### `box_cox`

In [13]:
def box_cox(predictors):
    '''Input dataframe to deskew it'''
    df_model_bc = pd.DataFrame()
    for col in predictors.columns:
        box_cox, lmbda = boxcox(predictors[col])
        df_model_bc[col] = pd.Series(box_cox)
    
    df_model_bc.set_index(predictors.index, inplace=True)
    
    return df_model_bc

In [14]:
df_model_skewnorm = box_cox(df_model_mm)

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:901: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


### Standardize Features

**`StandardScaler()` on All Features**

Switched from `MinMaxScaler()` in prior notebooks to `StandardScaler()`. I had run `StandardScaler()` on the prior notebooks initially and then later decided to try with `MinMaxScaler()`. While the predictive scores weren't changed by the switch, the `MinMaxScaler()` led to some odd features being deemed important by the feature selection. Therefore, I decided to stick with `StandardScaler()`.

In [16]:
standardized = (StandardScaler().fit_transform(df_model_skewnorm))
df_standardized = pd.DataFrame(standardized, columns=df_model_skewnorm.columns, index=df_model_skewnorm.index)

### Feature Selection

#### `SelectFromModel` with L1 penalty estimator

In [19]:
sfm = SelectFromModel(LogisticRegression(penalty='l1'), threshold='mean')
sfm.fit(df_standardized, target)

SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
        prefit=False, threshold='mean')

In [20]:
sfm_feats = np.where(sfm.get_support())[0]
sfm_feats

array([ 1,  2,  4,  8,  9, 18])

In [21]:
columns = list(df_standardized.columns)

sfm_feats_names = []
for i in sfm_feats:
    sfm_feats_names.append(columns[i])
    
sfm_feats_names

['ev_mph', 'dist', 'launch_angle', 'zone_13.0', 'zone_14.0', 'zone_unknown']

### Create dataframe with only selected features

In [22]:
df_slim = df_standardized[sfm_feats_names]

### `train_test_split` the data

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df_slim, target, test_size=.2, stratify=target)

### K Neighbors Classifier - manual modeling

In [25]:
knn = KNeighborsClassifier()

In [26]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [27]:
y_pred = knn.predict(X_test)

In [28]:
print(accuracy_score(y_test, y_pred))

0.803864468144


In [29]:
knn.score(X_test, y_test)

0.80386446814371726

In [30]:
print(f1_score(y_test, y_pred))

0.685432971472


### Define `run_benchmark` function to automate modeling

In [46]:
def run_benchmark(model, model_name, predictors, target):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, stratify=target)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return {'train_score' : model.score(X_train, y_train), 
            'test_score' : model.score(X_test, y_test),
            'kappa_score': cohen_kappa_score(y_pred, y_test),
            'model_name' : model_name }

# credit to Joshua Cook

### Run Models on Skew-Normalized and Standardized Data

Taking a look at `cohen_kappa_score()` to assess validity of handling class imbalance in subsequent step. Given highest values only as high as ~.53, I believe this step might help to hone predictive power.

### K Neighbors Classifier

In [53]:
knn_output = run_benchmark(KNeighborsClassifier(n_jobs=7), 
                               'kneighbors',
                               df_slim, 
                               target)

In [54]:
knn_output

{'kappa_score': 0.53729723678046937,
 'model_name': 'kneighbors',
 'test_score': 0.80184491389352386,
 'train_score': 0.85989988351226265}

### Logistic Regression

In [51]:
log_reg_output = run_benchmark(LogisticRegression(n_jobs=7), 
                               'logistic regression',
                               df_slim, 
                               target)

In [52]:
log_reg_output

{'kappa_score': 0.36794840163060061,
 'model_name': 'logistic regression',
 'test_score': 0.74728457639391743,
 'train_score': 0.74791423983880612}

### Decision Tree

In [49]:
dtree_output = run_benchmark(DecisionTreeClassifier(), 
                             'decision tree',
                             df_slim, 
                               target)

In [50]:
dtree_output

{'kappa_score': 0.45864075146518146,
 'model_name': 'decision tree',
 'test_score': 0.76205018417655768,
 'train_score': 0.99981110096653336}

### Random Forest Classifier

In [47]:
rand_forest_output = run_benchmark(RandomForestClassifier(n_jobs=7), 
                                  'random forest', 
                                  df_slim, 
                                  target)

In [48]:
rand_forest_output

{'kappa_score': 0.53268275199667081,
 'model_name': 'random forest',
 'test_score': 0.80367093788370114,
 'train_score': 0.98315650284922707}

### Show benchmark models side-by-side

In [41]:
output = [
    knn_output,
    log_reg_output,
    dtree_output, 
    rand_forest_output
]

pd.DataFrame(output)

,model_name,test_score,train_score
0,kneighbors,0.800491,0.859617
1,logistic regression,0.745112,0.748533
2,decision tree,0.763813,0.999780
3,random forest,0.801026,0.983408


As suspected, while normalizing the data in the earlier step drastically lowered the test score for kneighbors, after applying feature selection the test accuracy score has now improved to around 80% with significantly lower dimensionality. Random Forest is also around 80% accuracy.